In [112]:
#%% Import libraries
import numpy as np
from sympy import *
# from sympy.physics.mechanics import ReferenceFrame
from IPython.display import display, Latex, Math
init_printing(use_latex='mathjax')

from sys import path
path.append(
    "c:\\Users\\diego\\Dropbox\\Academic\\MEng Space Systems\\3. DOCA\\ADCS functions")
import ADCS_Functions as adcs

In [113]:
#%% Functions

def print_matrix(array, decimals=3):
    """
    A function to just print a matrix in Latex form. It just looks nicer.
    """
    matrix = ''
    for row in array:
        try:
            for number in row:
                matrix += f'{round(number,decimals)}&'
        except TypeError:
            matrix += f'{round(row,decimals)}&'
        matrix = matrix[:-1] + r'\\'
    display(Math(r'\begin{bmatrix}'+matrix+r'\end{bmatrix}'))


def char_poly(J):
    """
    Inputs a matrix in sympy form.
    Finds the characteristic polynomial of a matrix in sympy form.
    Takes the coefficients of the polynomial as a numpy array and Outputs the roots of the polynomial.
    NOTE: TBH I could also just use the find_eigen function or just numpy... But this way I get the characteristic polynomial.
    """
    # J.charpoly() gives the characteristic polynomial of J. Can also write as J.charpoly().as_expr() to just get the poly equation
    char_eq = J.charpoly()
    coef = np.array(char_eq.all_coeffs())
    return J.charpoly().as_expr(), np.roots(coef)


def find_eigen(J):
    """
    Input: a matrix in sympy form.
    Output: the eigenvalues and eigenvectors of a matrix in sympy form as numpy arrays
    """
    # J.eigenvects() gives the eigenvectors of J. Can also write as J.eigenvects().as_expr() to just get the eigenvectors
    Eigen = np.linalg.eigh(np.array(J, dtype='float'))
    Eigenvalues = Eigen[0]
    Eigenvectors = Eigen[1]
    return Eigenvalues, Eigenvectors


## Assume the rotational kinetic energy has the form: 

### $2 T_r = 20\omega_x^2 + 30\omega_y^2 + 15 \omega_z^2-20\omega_x\omega_y-30\omega_x\omega_z$


In [114]:
## Define symbols
T_r2 = Symbol('2T_r')
omega_x = Symbol('omega_x')
omega_y = Symbol('omega_y')
omega_z = Symbol('omega_z')

In [115]:
T_r2 = (20*omega_x**2 + 30*omega_y**2 + 15 *
                omega_z**2-20*omega_x*omega_y-30*omega_x*omega_z)
T_r2

     2                                 2         2
20⋅ωₓ  - 20⋅ωₓ⋅ω_y - 30⋅ωₓ⋅ω_z + 30⋅ω_y  + 15⋅ω_z 

## Please write the rotational kinetic energy in principle axis.

### Hint: The above rotational kinetic energy has the form $2 T_r = ω^T Jω$ . Find the appropriate transformation so that the rotational kinetic energy in principle axis has the form $2 T_r = ω^T Iω$ , where I is now a diagonal matrix.

The rotational kinetic energy in the form: $2 T_r = 20\omega_x^2 + 30\omega_y^2 + 15 \omega_z^2-20\omega_x\omega_y-30\omega_x\omega_z$
can be written in matrix notation as follows $2 T_r = ω^T Jω$. The associated non-diagonal inertia matrix has the following form:

Identifying the terms:

$\begin{equation}
2 T_{r}=\omega^{T} J \omega=J_{x x} \omega_{x}^{2}+J_{y y} \omega_{y}^{2}+J_{z z} \omega_{z}^{2}+2 J_{x y} \omega_{x} \omega_{y}+2 J_{y z} \omega_{y} \omega_{z}+2 J_{x z} \omega_{x} \omega_{z}
\end{equation}$

In [116]:
# In matrix form:
J = Matrix([[20, -10, -15], [-10, 30, 0], [-15, 0, 15]])

omegas = Matrix([omega_x, omega_y, omega_z])

omegas.T, J, omegas


⎛                ⎡20   -10  -15⎤  ⎡ωₓ ⎤⎞
⎜                ⎢             ⎥  ⎢   ⎥⎟
⎜[ωₓ  ω_y  ω_z], ⎢-10  30    0 ⎥, ⎢ω_y⎥⎟
⎜                ⎢             ⎥  ⎢   ⎥⎟
⎝                ⎣-15   0   15 ⎦  ⎣ω_z⎦⎠

In [117]:
# Check
simplify(omegas.T*J*omegas)

⎡     2                                 2         2⎤
⎣20⋅ωₓ  - 20⋅ωₓ⋅ω_y - 30⋅ωₓ⋅ω_z + 30⋅ω_y  + 15⋅ω_z ⎦

In [118]:
char_eq = J.charpoly()#.as_expr()
char_eq

PurePoly(lambda**3 - 65*lambda**2 + 1025*lambda - 750, lambda, domain='ZZ')

In [119]:
Eigenvalues, Eigenvectors = find_eigen(J)

print_matrix(Eigenvalues)

I = np.round(Eigenvalues, 2)*np.eye(3) # Principle moments of inertia
print_matrix(I)
print('I_1 = ', round(Eigenvalues[0], 2))
print('I_2 = ', round(Eigenvalues[1], 2))
print('I_3 = ', round(Eigenvalues[2], 2))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

I_1 =  0.77
I_2 =  24.65
I_3 =  39.58


In [120]:
print_matrix(Eigenvectors)  # Rotation matrix


<IPython.core.display.Math object>

In [121]:
# Rotation matrix. Need to transpose 
Rot_matrix = Eigenvectors.T
print_matrix(Rot_matrix)


<IPython.core.display.Math object>

In [122]:
# Check that the principle inertia matrix is correct
print_matrix(Rot_matrix @ np.array(J, dtype='float') @ Rot_matrix.T)


<IPython.core.display.Math object>

### The rotational kinetic energy can be simply written as $2 T_r = \bold{ω}^T \bold{Iω}$:

In [123]:
T_r2 = (I[0]*omega_x**2 + I[1]*omega_y**2 + I[2] *
              omega_z**2)
T_r2


array([0.77*omega_x**2, 24.65*omega_y**2, 39.58*omega_z**2], dtype=object)